<a href="https://colab.research.google.com/github/Jkt1412/Imdb-DataSet-Sentiment/blob/master/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [0]:
!ls

gdrive	sample_data


In [0]:
cd Imdb\ DataSet

/content/gdrive/My Drive/Colab Notebooks/Datasets/Imdb DataSet


In [0]:
cd gdrive/Colab\ Notebooks/Datasets/Imdb\ DataSet

[Errno 2] No such file or directory: 'gdrive/Colab Notebooks/Datasets/Imdb DataSet'
/content


In [0]:
cd gdrive

/content/gdrive


In [0]:
!ls

'My Drive'


In [0]:
cd My\ Drive/Colab\ Notebooks/Datasets/Imdb\ DataSet

/content/gdrive/My Drive/Colab Notebooks/Datasets/Imdb DataSet


In [0]:
!ls

aclImdb.rar


In [0]:
!pip install pyunpack

  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
  Stored in directory: /root/.cache/pip/wheels/41/22/19/af15ef6264c58b625a82641ed7483ad05e258fbd8925505227
Successfully built pyunpack easyprocess


In [0]:
!pip install patool

    100% |████████████████████████████████| 81kB 3.2MB/s 


In [0]:
from pyunpack import Archive
Archive('aclImdb.rar').extractall('./')

In [0]:
cd gdrive/My\ Drive/Colab\ Notebooks/Datasets/Imdb\ DataSet/aclImdb

/content/gdrive/My Drive/Colab Notebooks/Datasets/Imdb DataSet/aclImdb


## Actual Code Starts from here, Before this extracting RAR file and coming to the desired directory

In [1]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [2]:
cd gdrive/My\ Drive/Colab\ Notebooks/Datasets/Imdb\ DataSet/aclImdb

/content/gdrive/My Drive/Colab Notebooks/Datasets/Imdb DataSet/aclImdb


In [0]:
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

!cd aclImdb && mkdir movie_data

cd Imdb\ DataSet/aclImdb

!for split in test; do for sentiment in neg; do for file in $split/$sentiment/*; do cat $file >> movie_data/full_${split}.txt; echo >> movie_data/full_${split}.txt; done; done; done;

In [0]:
reviews_train = []
for line in open("./movie_data/full_train.txt",'r'):
  reviews_train.append(line.strip())
  
reviews_test = []
for line in open("./movie_data/full_test.txt",'r'):
  reviews_test.append(line.strip())

In [0]:
l1 = [[1.0,0.0]] * 12500
l2 = [[0.0,1.0]] * 12500
labels = l1 + l2

*   reviews_train_with_labels = np.array([[i,j] for i,j in zip(reviews_train,labels)])
*   reviews_test_with_labels = np.array([[i,j] for i,j in zip(reviews_test,labels)])

In [0]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess(reviews):
  reviews = [REPLACE_NO_SPACE.sub("",line.lower()) for line in reviews]
  reviews = [REPLACE_WITH_SPACE.sub(" ",line) for line in reviews]
  
  return reviews

In [0]:
reviews_clean_train = preprocess(reviews_train) 
reviews_clean_test = preprocess(reviews_test) 

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
lemmatized_review_train = []
lemmatized_review_test = []
for review in reviews_clean_train:
  tokenized_text = tokenizer.tokenize(review)
  lemmatized_text = [lemmatizer.lemmatize(words) for words in tokenized_text]
  lemmatized_string = " ".join(x for x in lemmatized_text)
  lemmatized_review_train.append(lemmatized_string)
  
for review in reviews_clean_test:
  tokenized_text = tokenizer.tokenize(review)
  lemmatized_text = [lemmatizer.lemmatize(words) for words in tokenized_text]
  lemmatized_string = " ".join(x for x in lemmatized_text)
  lemmatized_review_test.append(lemmatized_string)
  
  

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf_train = TfidfVectorizer(min_df = 100, max_df = 14000, ngram_range = (1,2))

In [0]:
features_train_idf = tfidf_train.fit_transform(lemmatized_review_train)

In [13]:
pd.DataFrame(features_train_idf.todense(), columns = tfidf_train.get_feature_names()).head()

,10,10 10,10 minute,10 out,10 star,10 year,100,11,12,13,...,youre looking,youre not,yourself,youth,youve,youve got,youve seen,zero,zombie,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
training_features = np.array(features_train_idf.todense())

In [0]:
def next_batch(features,batch_size):
  random = np.random.randint(0,25000-batch_size-1)
  return features[random:random+batch_size] , np.array(labels[random:random+batch_size])

In [0]:
tf.reset_default_graph()

In [0]:
num_batches = 1000
num_hidden_neurons = 50
learning_rate = 0.3
num_features_train = int(training_features.shape[1])

In [0]:
X = tf.placeholder(tf.float32,shape = [None,num_features_train])
y_true = tf.placeholder(tf.float32,shape = [None,2])

In [0]:
from tensorflow.contrib.layers import fully_connected

In [0]:
output = fully_connected(X,num_outputs=2,activation_fn = tf.nn.sigmoid)

In [0]:
#cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels = y_true,logits = output)
mse = tf.reduce_mean(tf.square(output-y_true))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
train = optimizer.minimize(mse)

In [0]:
init = tf.global_variables_initializer()

In [56]:
with tf.Session() as sess:
  sess.run(init)
  
  iterations = 10000
  for i in range(iterations):
    batch_x,batch_y = next_batch(training_features,num_batches)
    sess.run(train,feed_dict = {X:batch_x,y_true:batch_y})
      
    if i%2500 == 0:
      matches = tf.equal(tf.argmax(y_true,1),tf.argmax(output,1))
      acc = tf.reduce_mean(tf.cast(matches,tf.float32))
      accuracy = sess.run(acc,feed_dict ={X:training_features,y_true:np.array(labels)})
      print("Accuracy is: ")
      print(accuracy)
      print("\n")
  
  

Accuracy is: 
0.5


Accuracy is: 
0.74664


Accuracy is: 
0.8256


Accuracy is: 
0.8372


